**LANGGRAPH 101**

Langgraph is a framework for building agents
Has three components:
1. Langchain- LLm Intergration
2. LangSmith- Observability(Tracing and Evaluation)
3. LangGraph- Application workflow


**Chat Models**
Chat models are the foundation of LLM application. Include GPT, Llama, Gemini, Deepseek.


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

#load environment variables
load_dotenv()


api_key=os.getenv("GITHUB_TOKEN")

llm=ChatOpenAI(
    model="gpt-4o",
    openai_api_key=api_key,
    base_url="https://models.inference.ai.azure.com"
)



**Running the Model**

Methods for using the chat models
1. invoke(): A single input is transformed into an output.
2. stream(): Outputs are streamed as they are produced.


In [3]:
result=llm.invoke("What is an agent?")


In [4]:
type(result)

langchain_core.messages.ai.AIMessage

In [5]:
from rich.markdown import Markdown
Markdown(result.content)

An "agent" is a versatile term that can be interpreted in various contexts, depending on the subject matter. Below 
are some common definitions across different fields:                                                               

                                              1. General Definition:                                               

An agent is an entity that acts on behalf of someone or something, often performing specific tasks or achieving    
particular goals.                                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             2. In Artificial Intelligence (AI) and Computer Science:                              

An agent is a system or software that perceives its environment through sensors, acts on that environment through  
actuators, and aims to perform actions to meet specific objectives. Examples include:                              

 • Autonomous Agents: Systems like self-driving cars or AI chatbots.                                               
 • Intelligent Agents: AI systems designed to learn, adapt, and make decisions.                                    
 • Software Agents: Programs that perform automated tasks, such as web crawlers or recommendation systems.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      3. In Philosophy and Cognitive Science:                                      

An agent refers to an entity with the capacity to act, make decisions, and exert influence, often with             
intentionality and autonomy.                                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     4. In Business, Law, and Representation:                                      

An agent is someone authorized to represent another person or organization in dealings with third parties.         
Examples:                                                                                                          

 • Talent Agent: Represents actors, writers, or athletes to negotiate contracts.                                   
 • Insurance Agent: Sells insurance policies on behalf of an insurance company.                                    
 • Real Estate Agent: Facilitates the buying, selling, or leasing of properties.                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                         5. In Chemistry and Biosciences:                                          

An agent is a substance that produces a reaction or has a specific function. Examples include:                     

 • Catalyst Agent: Speeds up a chemical reaction.                                                                  
 • Therapeutic Agent: A drug or compound used in medicine to treat conditions.                                     
 • Cleaning Agent: A chemical used for cleaning purposes.                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                           6. In Security and Espionage:                                           

An agent refers to an individual who collects intelligence or carries out missions, often covertly. Examples       
include:                                                                                                           

 • Secret Agent: Works for a spy agency (e.g., James Bond in fiction).                                             
 • Double Agent: Acts as an agent for tw

**Tools**

Tools are utilities that can be called by a chat model.
We use @tool decorator to create a tool, which transforms python functions into a callable tools.


In [6]:
from langchain.tools import tool

@tool
def write_email(to: str, subject:str, content:str)->str:
    """Write and send an email"""
    #placeholder response in real app would send email
    return f"Email Sent to {to} with subject '{subject}' and content: {content}"

In [7]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [8]:
#check the argument the tool requires
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [ ]:
#view the tool description
Markdown(write_email.description)

Write and send an email

**Tool Calling**

Tools can be called by LLMs. When a tool is binded to a model it can choose to call the tool by returning a structured output with tool arguments.
 
**bind_tools** allows us to give an llm access to tools

**"Any"** will select at least one of the tools

In addition we can set **parallel_tool_calls=False** to ensure the model will only call one tool at a time.


In [10]:
#connect tools to a chat model
model_with_tools=llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)


#The model will now be able to call tools
output=model_with_tools.invoke("Draft a response to my boss ('boss@company.ai') about tomorrow's meeting")

In [11]:
type(output)

langchain_core.messages.ai.AIMessage

In [12]:
output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aAepvT4ZPe5pXYUDR5dfyvZC', 'function': {'arguments': '{"to":"boss@company.ai","subject":"Re: Tomorrow\'s Meeting","content":"Dear [Boss\'s Name],\\n\\nThank you for the update regarding tomorrow\'s meeting. Could you please confirm the exact time and location, as well as any specific topics or materials I should prepare in advance?\\n\\nLooking forward to the discussion.\\n\\nBest regards,\\n[Your Name]"}', 'name': 'write_email'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 67, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-Bqlkb3y5wVa0u7N9135Wf1xtjoSSP', 'service_tier': None, 'fin

In [14]:
#extract tool calls and execute them
args=output.tool_calls[0]['args']
args

{'to': 'boss@company.ai',
 'subject': "Re: Tomorrow's Meeting",
 'content': "Dear [Boss's Name],\n\nThank you for the update regarding tomorrow's meeting. Could you please confirm the exact time and location, as well as any specific topics or materials I should prepare in advance?\n\nLooking forward to the discussion.\n\nBest regards,\n[Your Name]"}

In [15]:
#call the tool
result=write_email.invoke(args)
Markdown(result)

Email Sent to boss@company.ai with subject 'Re: Tomorrow's Meeting' and content: Dear [Boss's Name],               

Thank you for the update regarding tomorrow's meeting. Could you please confirm the exact time and location, as    
well as any specific topics or materials I should prepare in advance?                                              

Looking forward to the discussion.                                                                                 

Best regards, [Your Name]

**Workflows**

There are many patterns for building applications with LLMs.

We can embed LLM calls into pre-defined workflows, giving the system more agency to make decisions.

E.G Add a router step to deteremine whether to write an email or not.


**Agents**

They further increase agency allowing the LLM to dynamically direct its own tool usage.

Output of each tool called is used to inform the next action.

